In [43]:
import pandas as pd
import numpy as np
#import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
import matplotlib as plt
import seaborn as sns
from tqdm.auto import tqdm

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [3]:
df = pd.read_csv('course_lead_scoring.csv')

df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


# Deal with Null Values

## Data preparation

Check if the missing values are presented in the features.

If there are missing values:

For caterogiral features, replace them with 'NA'

For numerical features, replace with with 0.0

Split the data into 3 parts: train/validation/test with 60%/20%/20% distribution. Use 

train_test_split function for that with random_state=1

In [28]:
## Categorical Values

categorical = df.dtypes[df.dtypes=='object'].index.to_list()
categorical

['lead_source', 'industry', 'employment_status', 'location']

In [21]:
## Numerical values
numerical = df.dtypes[(df.dtypes=='int64')|(df.dtypes=='float64')].index.to_list()

numerical

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score',
 'converted']

In [32]:
## these are all categorical Null Values
df.loc[:,categorical].isnull().sum()

lead_source          128
industry             134
employment_status    100
location              63
dtype: int64

In [38]:
## Fillna() on the subset of columns that are categorical
df[categorical] = df[categorical].fillna('NA')

In [41]:
## Fillna() on the subset of columns that are numerical
df[numerical] = df[numerical].fillna(0.0)

df[numerical].isnull().sum()

number_of_courses_viewed    0
annual_income               0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

## Train Test Split 

Split the data into 3 parts: train/validation/test with 60%/20%/20% distribution. 

Use train_test_split function for that with random_state=1

In [44]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

# QUESTION 1 ROC AUC feature importance


In [46]:
from sklearn.metrics import roc_auc_score


In [47]:
df[numerical].head(3)

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
0,1,79450.0,4,0.94,1
1,1,46992.0,1,0.80,0
2,5,78796.0,3,0.69,1


In [48]:
## calculate ROC AUC score for each numerical feature using training set
roc_auc_score(y_train, df_train['number_of_courses_viewed'])

0.7635680590007088

In [55]:
## numerical columns excluding "y" variable "converted"
df[numerical].drop(columns='converted').columns.to_list()

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [59]:
## For Loop to calculate every numerical feature's ROC AUC score
auc_dict = {}
for col in df[numerical].drop(columns='converted').columns.to_list():
    auc_score = roc_auc_score(y_train, df_train[col])
    auc_dict[col] = auc_score

## Store in DataFrame
df_auc = pd.DataFrame.from_dict(auc_dict, orient='index', columns=['AUC']).sort_values(by='AUC', ascending=False)
df_auc

,AUC
number_of_courses_viewed,0.763568
interaction_count,0.738270
lead_score,0.614499
annual_income,0.551958


### QUESTION 1 ANSWER = number of courses viewed has highest ROC AUC score